In [1]:
    import os
    import numpy as np
    import pandas as pd
    import xarray as xr
    import matplotlib.pyplot as plt
    from matplotlib import colors
    from eofs.xarray import Eof
    from esem import gp_model
    import seaborn as sns
    import cartopy.crs as ccrs
    from gp_helpers_from_reproduction import *

    import warnings

    warnings.filterwarnings("ignore")

C:\Users\16195\AppData\Local\Temp\ipykernel_80312\1374887712.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
files = ["ssp126", "ssp585", "historical", "hist-GHG"]
# Create training and testing arrays
X_train, pca_solvers = makeXTrain_PCASolvers(files)
y_train_tas = makeYTrain(files)['tas'].values.reshape(-1, 96 * 144)

X_test = makeXTest('ssp245', pca_solvers)
Y_test = xr.open_dataset(data_path + 'outputs_ssp245.nc').compute()
tas_truth = Y_test["tas"].mean('member')

In [3]:
# Removing NaNs 
nan_train_mask, nan_test_mask = X_train.isna().any(axis=1).values, X_test.isna().any(axis=1).values
X_train, X_test = X_train.dropna(axis=0, how='any'), X_test.dropna(axis=0, how='any')
y_train_tas, tas_truth = y_train_tas[~nan_train_mask], tas_truth[~nan_test_mask]

In [4]:
# Standardizing data for data that is not EOFs/PCA
X_train_CO2_mean, X_test_CO2_mean = X_train['CO2'].mean(), X_test['CO2'].mean()
X_train_CO2_std, X_test_CO2_std = X_train['CO2'].std(), X_test['CO2'].std()

X_train_CH4_mean, X_test_CH4_mean = X_train['CH4'].mean(), X_test['CH4'].mean()
X_train_CH4_std, X_test_CH4_std = X_train['CH4'].std(), X_test['CH4'].std()

X_train['CO2'] = (X_train['CO2'] - X_train_CO2_mean) / X_train_CO2_std
X_test['CO2'] = (X_test['CO2'] - X_test_CO2_mean) / X_test_CO2_std

X_train['CH4'] = (X_train['CH4'] - X_train_CH4_mean) / X_train_CH4_std
X_test['CH4'] = (X_test['CH4'] - X_test_CH4_mean) / X_test_CH4_std

# Standardizing y_train
y_train_tas_mean, y_train_tas_std = y_train_tas.mean(), y_train_tas.std()
y_train_tas = (y_train_tas - y_train_tas_mean) / y_train_tas_std

# Model

In [5]:
X_train.shape, y_train_tas.shape

((502, 12), (502, 13824))

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt

import gpflow
import gpflux

from gpflow.config import default_float


In [7]:
num_data = len(X_train)
num_inducing = 10
output_dim = y_train_tas.shape[1]

kernel = gpflow.kernels.SquaredExponential()
inducing_variable = gpflow.inducing_variables.InducingPoints(
    np.linspace(X_train.min(), X_train.max(), num_inducing).reshape(-1, 1)
)
gp_layer = gpflux.layers.GPLayer(
    kernel, inducing_variable, num_data=num_data, num_latent_gps=output_dim
)

In [ ]:
likelihood = gpflow.likelihoods.Gaussian(0.1)

# So that Keras can track the likelihood variance, we need to provide the likelihood as part of a "dummy" layer:
likelihood_container = gpflux.layers.TrackableLayer()
likelihood_container.likelihood = likelihood

model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(1, activation="linear"),
        gp_layer,
        likelihood_container,  # no-op, for discovering trainable likelihood parameters
    ]
)
loss = gpflux.losses.LikelihoodLoss(likelihood)

In [ ]:
model.compile(loss=loss, optimizer="adam")

In [ ]:
hist = model.fit(X_train, y_train_tas, epochs=500, verbose=1)
plt.plot(hist.history["loss"])